# Imports

In [ ]:
# Suprress warnings
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    # Standard imports
    import matplotlib
    import matplotlib.pyplot as plt
    import matplotlib.patches as patches
    import matplotlib.cm as cm
    import numpy as np

    # Drake imports
    from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
    import pydrake
    from pydrake.all import LogOutput, DirectCollocation, DirectTranscription, MathematicalProgram, InputPortSelection
    from pydrake.all import FindResourceOrThrow
    
    import scipy.interpolate
# Imports of other project files
from log_wrapper import LogWrapper
import arm
import constants
import finger
import pedestal
from paper import Paper

from pydrake.all import (MultibodyPlant, Parser, DiagramBuilder, Simulator, RigidTransform,
                         PlanarSceneGraphVisualizer, SceneGraph, TrajectorySource,
                         SnoptSolver, MultibodyPositionToGeometryPose, PiecewisePolynomial,
                         MathematicalProgram, JacobianWrtVariable, eq, RollPitchYaw, AutoDiffXd, BodyIndex,
                        RotationMatrix, Meshcat,MeshcatVisualizerParams, MeshcatVisualizerCpp, MeshcatVisualizer)

# Other imports
import importlib

In [ ]:
from IPython.display import display, SVG
import pydot

In [ ]:
# Matplotlib configuring
# USE FOR PAPER
# plt.style.use(['science', 'no-latex'])
# font = {'size'   : 14}
# matplotlib.rc('font', **font)
# default_figsize = (2*3,2*2)
# USE GENERALLY
font = {'size'   : 16}
matplotlib.rc('font', **font)
default_figsize = (16,8)

## Meshcat init (including link for new meshcat)

In [ ]:
if constants.USE_NEW_MESHCAT:
    meshcat = Meshcat()
    web_url = meshcat.web_url()
else:
    proc, zmq_url, web_url = start_zmq_server_as_subprocess()

# Simulation setup
## Pre-finalize steps

In [ ]:
builder = pydrake.systems.framework.DiagramBuilder()

jnt_frc_log = []
arm_acc_log = []
arm_acc_log.append(np.zeros(7))

# Add all elements
plant, scene_graph = pydrake.multibody.plant.AddMultibodyPlantSceneGraph(builder, time_step=constants.DT)
v_stiction=1e-3
plant.set_stiction_tolerance(v_stiction)
plant.set_penetration_allowance(0.001)
pedestal_instance = pedestal.AddPedestal(plant)

# These joint angles start the paper approximately the right spot, no matter how many links are used
num_links = 2
def_joint_angles = 0

paper = Paper(plant, scene_graph, num_links, default_joint_angle=def_joint_angles,
              stiffness=2.5e-1,
              damping=7.12547340446979e-06)
paper.weld_paper_edge(pedestal.PEDESTAL_WIDTH, pedestal.PEDESTAL_HEIGHT)

Calculate link inertia to use in damping calculations:

In [ ]:
arm_instance = arm.AddArm(plant, scene_graph)

In [ ]:
# Set up logger (needs to happen after all bodies are added)
top_finger_body = plant.GetBodyByName(arm.TOP_FINGER_NAME)
bot_finger_body = plant.GetBodyByName(arm.BOT_FINGER_NAME)
log_wrapper = LogWrapper(plant.num_bodies(), int(bot_finger_body.index()), int(top_finger_body.index()), paper, jnt_frc_log, arm_acc_log)
builder.AddSystem(log_wrapper)

## Controller selection

In [ ]:
ll_idx = paper.link_idxs[-1]
top_finger_idx = int(top_finger_body.index())
bot_finger_idx = int(bot_finger_body.index())

In [ ]:
I_L = plant.get_body(
    BodyIndex(paper.link_idxs[-1])).default_rotational_inertia().CalcPrincipalMomentsOfInertia()[0]
sys_params = {
    'I_L': I_L,
    'v_stiction': v_stiction,
    'w_L': paper.link_width,
    'm_L': paper.link_mass,
    'b_J': paper.damping,
    'k_J': paper.stiffness,
    'g': plant.gravity_field().gravity_vector()[-1]*-1,
}
fold_ctrl = arm.ArmFoldingController(
    ll_idx=int(paper.link_idxs[-1]),
    sys_params=sys_params,
    jnt_frc_log=jnt_frc_log,
    options={
        'use_friction_adaptive_ctrl': False,
        'use_friction_robust_adaptive_ctrl': False,
    },
    arm_acc_log=arm_acc_log,
)

## Post-finalize steps

In [ ]:
plant.Finalize()

In [ ]:
## Post finalize steps
# Add logger
builder.Connect(plant.get_body_poses_output_port(), log_wrapper.get_input_port(0))
builder.Connect(plant.get_body_spatial_velocities_output_port(), log_wrapper.get_input_port(1))
builder.Connect(plant.get_body_spatial_accelerations_output_port(), log_wrapper.get_input_port(2)) 
builder.Connect(plant.get_contact_results_output_port(), log_wrapper.get_input_port(3))
builder.Connect(plant.get_reaction_forces_output_port(), log_wrapper.get_input_port(4))
builder.Connect(plant.get_generalized_acceleration_output_port(arm_instance), log_wrapper.get_input_port(5))
builder.Connect(plant.get_state_output_port(arm_instance), log_wrapper.get_input_port(6))

# Add arm controller
builder.AddSystem(fold_ctrl)
builder.Connect(plant.get_state_output_port(arm_instance), fold_ctrl.get_input_port(0))
builder.Connect(plant.get_body_poses_output_port(), fold_ctrl.get_input_port(1))
builder.Connect(plant.get_body_spatial_velocities_output_port(), fold_ctrl.get_input_port(2))
builder.Connect(plant.get_contact_results_output_port(), fold_ctrl.get_input_port(3))

builder.Connect(fold_ctrl.get_output_port(), plant.get_actuation_input_port())

# Visualization and logging
logger = LogOutput(log_wrapper.get_output_port(), builder)

if constants.USE_NEW_MESHCAT:
    params = MeshcatVisualizerParams()
    vis = MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph.get_query_output_port(), meshcat, params)
    fold_ctrl.set_meshcat(meshcat)
else:
    vis = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder, scene_graph)
    # vis.set_planar_viewpoint(camera_position=[1, 0, 0], xmin=-0.3, xmax=0.3, ymin=-0.3, ymax=0.3)

# Build diagram and do actions requiring 
diagram = builder.Build()
diagram_context = diagram.CreateDefaultContext()

## Meschat link for old meschat

In [ ]:
q0 = np.zeros(7)
q0[0] = -np.pi/2
q0[1] = 0# 1.1
q0[3] = np.pi/2
q0[5] = np.pi/2#3*np.pi/2 + 0.1
q0[6] = -np.pi/4
plant_context = diagram.GetMutableSubsystemContext(plant, diagram_context)
plant.SetPositions(plant_context, arm_instance, q0)

In [ ]:
# SVG(pydot.graph_from_dot_data(diagram.GetGraphvizString())[0].create_svg())

# Run simulation

In [ ]:
# Finalize simulation and visualization
simulator = pydrake.systems.analysis.Simulator(diagram, diagram_context)
simulator.Initialize()
if not constants.USE_NEW_MESHCAT:
    vis.start_recording()
try:
    simulator.AdvanceTo(constants.TSPAN)
except RuntimeError as e:
    print(e)
except np.linalg.LinAlgError as e:
    print(e)

In [ ]:
# This way, we can use the actual time the simulation ran for
effective_tspan = max(logger.sample_times())
print("effective_tspan:", effective_tspan)

if not constants.USE_NEW_MESHCAT:
    vis.stop_recording()
    vis.publish_recording()

# Plots used in orginal paper

In [ ]:
# This script is getting huge. We need to turn off parts that we're not using.
generate_paper_plots = False

In [ ]:
if generate_paper_plots:
    # Generate plots for paper
    nb = plant.num_bodies()
    y_traces = []
    z_traces = []
    theta_traces = []
    for b in paper.link_idxs:
        y_traces.append(logger.data()[log_wrapper.entries_per_body*b+1])
        z_traces.append(logger.data()[log_wrapper.entries_per_body*b+2])
        theta_traces.append(logger.data()[log_wrapper.entries_per_body*b+3])
    y_traces = np.array(y_traces)
    z_traces = np.array(z_traces)
    theta_traces = np.array(theta_traces)

    times_ = np.arange(0,effective_tspan, effective_tspan/10)
    cmap = cm.get_cmap("viridis_r")
    plt.figure(figsize=(2*3,2*2))
    for t in times_:
        c = cmap(t/effective_tspan)
        idx = np.argmax(logger.sample_times() >= t)

        # Plot paper
        # PROGRAMMING: Account for paper thickness in plots
        for y, z, theta in zip(y_traces[:,idx], z_traces[:,idx], theta_traces[:,idx]):
            y0 = y - np.cos(theta)*paper.link_width/2
            z0 = z - np.sin(theta)*paper.link_width/2
            y1 = y + np.cos(theta)*paper.link_width/2
            z1 = z + np.sin(theta)*paper.link_width/2
            plt.plot([y0, y1], [z0, z1], color=c)


        # Plot manipulator
        plt.scatter(logger.data()[log_wrapper.entries_per_body*top_finger_idx+1,idx], 
                    logger.data()[log_wrapper.entries_per_body*top_finger_idx+2,idx],
                    color=c, s=300, zorder=1)

    xlim = plt.xlim()
    ylim = plt.ylim()
    plt.scatter([xlim[0]-50, xlim[0]-50], [ylim[0]-50, ylim[0]-50], c=[0, effective_tspan], cmap=cmap)
    plt.xlim(xlim)
    plt.ylim(ylim)
    cb = plt.colorbar()
    cb.set_label("Time")
    plt.xlabel("$y$ position")
    plt.ylabel("$z$ position")
    plt.show()

# Drag plots

This is a sanity check to see how much force we're neglecting by ignoring drag. The colored lines are the drag forces at each link, and the dashed black line is the force due to gravity for scale.

Note that the formula for drag force is $\rho v^2 C_d A_{paper}$.

$C_d$ comes from [this site](https://www.engineersedge.com/fluid_flow/rectangular_flat_plate_drag_14036.htm), which says it depends on $L/d$. For 20 links, $L/d\approx15$. Rounding this down to 10, we get $C_d=1.22$.

In [ ]:
generate_drag_plots = False

In [ ]:
if generate_drag_plots:
    # Plot drag forces
    plt.figure(figsize=(16, 12))
    nb = plant.num_bodies
    vel_traces = []
    for b in paper.link_idxs:
        rho = 1.225
        C_d = paper.link_width*paper.depth
        vel_sqred = logger.data()[log_wrapper.entries_per_body*b+7]**2+logger.data()[log_wrapper.entries_per_body*b+8]**2
        f = 0.5*rho*C_d*vel_sqred
        plt.plot(f)

    plt.axhline(paper.link_mass*9.81, color='black', linestyle='--')
    plt.show()

# Verifying equations hold
We have the following free body diagrams:
<img src="forces.png" width=400px>
<img src="link-moments.png" width=400px>

Which give us the following equations:
$$
\begin{aligned}
     m_La_{LT} &= F_{FL} + F_{GT} +F_{OT} \\
    m_La_{LN} &= F_{NL} + F_{GN} +F_{ON} \\
    m_Ma_{MT} &= F_{FM} + F_{CT} \\
    m_Ma_{MN} &= F_{NM} + F_{CN} \\
    a_{LT} &= a_{MT} \\
    a_{LN} &= a_{MN} \\
    a_{LT} &= -\frac{w_L}{2}\dot\theta^2 \\
    a_{LN} &= \frac{w_L}{2}\ddot\theta \\
    I_L\ddot\theta &= \frac{w_L}{2}F_{ON}- \frac{h_L}{2}F_{FL} - r_TF_{NL} \\
    F_{NL} &= -F_{NM} \\
    F_{FL} &= -F_{FM} \\
    a_{NL} &= a_{Nd}  \\
    F_{FL} &= 0
\end{aligned}
$$
For each of these equations, I want to plot the value on the left side and the value on the right side to see if they match.

# Loading signals
Preparing all the signals we'll need.

In [ ]:
debug = fold_ctrl.debug

In [ ]:
min_key_length = np.inf
for k in debug.keys():
    if len(debug[k]) > 0 and len(debug[k]) < min_key_length:
        min_key_length = len(debug[k])
for k in debug.keys():
    debug[k] = debug[k][:min_key_length]

In [ ]:
l_times = logger.sample_times()
d_times = np.array(debug['times'])

## Helper functions

In [ ]:
def diff_vec(vec):
    d_vec = np.diff(vec, axis=0)
    for i in range(vec.shape[1]):
        d_vec[:, i] /= np.expand_dims(np.diff(l_times), 1)
    d_vec = np.concatenate((d_vec, [d_vec[-1]]))
    return d_vec

In [ ]:
def plot_vec(sig1, sig1_label, sig2, sig2_label, finish_plotting=True, plot_x=True, start_time=None):
    if start_time is None:
        idx_start_ = idx_start
    else:
        idx_start_ = np.argmax(l_times > start_time)
    plt.figure(figsize=(16, 8))
    lw = 5
    if plot_x:
        plt.plot(l_times[idx_start_:idx_end], sig1[:,0][idx_start_:idx_end], label=sig1_label + r' $\hat x$ component', color='pink', linewidth=lw)
    plt.plot(l_times[idx_start_:idx_end], sig1[:,1][idx_start_:idx_end], label=sig1_label + r' $\hat y$ component', color='lightgreen', linewidth=lw)
    plt.plot(l_times[idx_start_:idx_end], sig1[:,2][idx_start_:idx_end], label=sig1_label + r' $\hat z$ component', color='lightskyblue', linewidth=lw)
    
    if plot_x:
        plt.plot(l_times[idx_start_:idx_end], sig2[:,0][idx_start_:idx_end], label=sig2_label + r' $\hat x$ component', color='red', linewidth=lw, linestyle='--')
    plt.plot(l_times[idx_start_:idx_end], sig2[:,1][idx_start_:idx_end], label=sig2_label + r' $\hat y$ component', color='green', linewidth=lw, linestyle='--')
    plt.plot(l_times[idx_start_:idx_end], sig2[:,2][idx_start_:idx_end], label=sig2_label + r' $\hat z$ component', color='blue', linewidth=lw, linestyle='--')
    plt.xlabel("Time (seconds)")
    plt.legend()
    plt.autoscale(enable=True, axis='x', tight=True)
    if finish_plotting:
        plt.show()

In [ ]:
def plot_vec_XTN(sig1, sig1_label, sig2, sig2_label, finish_plotting=True, plot_x=True, start_time=None):
    if start_time is None:
        idx_start_ = idx_start
    else:
        idx_start_ = np.argmax(l_times > start_time)
    plt.figure(figsize=(16, 8))
    lw = 5
    
    T_proj1 = np.matmul(T(sig1), T_hat).flatten()
    T_proj2 = np.matmul(T(sig2), T_hat).flatten()
    N_proj1 = np.matmul(T(sig1), N_hat).flatten()
    N_proj2 = np.matmul(T(sig2), N_hat).flatten()
    
    plt.plot(l_times[idx_start_:idx_end], sig1[:,0][idx_start_:idx_end], label=sig1_label + r' $\hat x$ component', color='pink', linewidth=lw)
    plt.plot(l_times[idx_start_:idx_end], T_proj1[idx_start_:idx_end], label=sig1_label + r' $\hat T$ component', color='lightgreen', linewidth=lw)
    plt.plot(l_times[idx_start_:idx_end], N_proj1[idx_start_:idx_end], label=sig1_label + r' $\hat N$ component', color='lightskyblue', linewidth=lw)
    
    plt.plot(l_times[idx_start_:idx_end], sig2[:,0][idx_start_:idx_end], label=sig2_label + r' $\hat x$ component', color='red', linewidth=lw, linestyle='--')
    plt.plot(l_times[idx_start_:idx_end], T_proj2[idx_start_:idx_end], label=sig2_label + r' $\hat T$ component', color='green', linewidth=lw, linestyle='--')
    plt.plot(l_times[idx_start_:idx_end], N_proj2[idx_start_:idx_end], label=sig2_label + r' $\hat N$ component', color='blue', linewidth=lw, linestyle='--')
    plt.xlabel("Time (seconds)")
    plt.legend()
    plt.autoscale(enable=True, axis='x', tight=True)
    if finish_plotting:
        plt.show()

In [ ]:
def diff_arr(arr):
    d_arr = np.diff(arr)/np.diff(l_times)
    d_arr = np.concatenate((d_arr, [d_arr[-1]]))
    return d_arr

In [ ]:
# Friction helper functions
def step5(x):
    '''Python version of MultibodyPlant::StribeckModel::step5 method'''
    x3 = x * x * x
    return x3 * (10 + x * (6 * x - 15))

def stribeck(us, uk, v):
    '''
    Python version of MultibodyPlant::StribeckModel::ComputeFrictionCoefficient
    
    From
    https://github.com/RobotLocomotion/drake/blob/b09e40db4b1c01232b22f7705fb98aa99ef91f87/multibody/plant/images/stiction.py
    '''
    u = np.zeros_like(v) + uk
    u[v < 1] = us * step5(v[v < 1])
    mask = (v >= 1) & (v < 3)
    u[mask] = us - (us - uk) * step5((v[mask] - 1) / 2)
    return u

In [ ]:
def T(vec):
    return np.transpose(vec, [0, 2, 1])

## Inputs
These should all come from simulation/geometry, since they are inputs to the controller and are "fixed" from the controller's perspective.
### Geometric quantities ($m_L, m_M, w_L, I_L, h_L, \mu$)

In [ ]:
m_L = paper.link_mass
w_L = paper.link_width
I_L = paper.plant.get_body(
    BodyIndex(paper.link_idxs[-1])).default_rotational_inertia().CalcPrincipalMomentsOfInertia()[0]
h_L = paper.height
mu_paper = constants.FRICTION
mu = 2*mu_paper/(1+mu_paper) # 2μₘμₙ/(μₘ + μₙ)
b_J = paper.damping
k_J = paper.stiffness

### Gravity, unit vectors ($F_{GT}, F_{GN}, \hat T, \hat N$)

In [ ]:
# Assume plane contact, so all N_hat should point the same direction
# TODO: verify all point in the same direction
N_hat_raw = logger.data()[log_wrapper.top_contact_entry_start_idx+14:log_wrapper.top_contact_entry_start_idx+17,:].copy() # So it's writeable
N_hat = np.expand_dims(N_hat_raw.T, 2)
N_hat[:,0][np.isnan(N_hat[:,0])] = 0
N_hat[:,1][np.isnan(N_hat[:,1])] = 0
N_hat[:,2][np.isnan(N_hat[:,2])] = 1

In [ ]:
T_hat = np.matmul(
    np.array([
        [1,  0, 0],
        [0,  0, 1],
        [0, -1, 0],
    ]),
    N_hat)

It's also useful here to calculate project matrices:

In [ ]:
N_hat_proj_mat = np.matmul(N_hat, np.transpose(N_hat, [0, 2, 1]))

In [ ]:
g = plant.gravity_field().gravity_vector()[-1]*-1
F_G = np.zeros([l_times.size, 3, 1])
F_G[:,2,0] = -np.ones_like(l_times)*g*m_L
F_GT = np.matmul(T(F_G), T_hat).flatten()
F_GN = np.matmul(T(F_G), N_hat).flatten()

In [ ]:
tau_g_no_interp = np.array(fold_ctrl.debug['tau_g'])
tau_g = np.expand_dims(scipy.interpolate.interp1d(fold_ctrl.debug['times'], tau_g_no_interp, axis=0)(l_times), 2)

### Positions, velocities ($\dot\theta^2, r_T, \dot d_T$)

#### Object poses & vels

In [ ]:
p_L = np.expand_dims(
    logger.data()[log_wrapper.entries_per_body*paper.link_idxs[-1]:
                  log_wrapper.entries_per_body*paper.link_idxs[-1]+3].T,
    2)
p_LN = np.matmul(T(p_L), T_hat).flatten()
p_LT = np.matmul(T(p_L), N_hat).flatten()
p_LX = p_L[:,0].flatten()

In [ ]:
theta_L = logger.data()[log_wrapper.entries_per_body*paper.link_idxs[-1] + 3]
theta_M = logger.data()[log_wrapper.entries_per_body*int(top_finger_body.index()) + 3]

In [ ]:
d_theta_L = logger.data()[log_wrapper.entries_per_body*paper.link_idxs[-1] + 6 + 3]
d_theta_M = logger.data()[log_wrapper.entries_per_body*int(top_finger_body.index()) + 6 + 3]

In [ ]:
x_hat = np.zeros((len(l_times), 3, 1))
x_hat[:,0,:] = 1

y_hat = np.zeros((len(l_times), 3, 1))
y_hat[:,1,:] = 1

z_hat = np.zeros((len(l_times), 3, 1))
z_hat[:,2,:] = 1

In [ ]:
v_L = np.expand_dims(
    logger.data()[log_wrapper.entries_per_body*paper.link_idxs[-1]+6:
                  log_wrapper.entries_per_body*paper.link_idxs[-1]+6+3].T,
    2)
v_LN = np.matmul(T(v_L), N_hat).flatten()
v_LT = np.matmul(T(v_L), T_hat).flatten()
v_LX = v_L[:,0].flatten()

In [ ]:
p_M_top = np.expand_dims(logger.data()[log_wrapper.entries_per_body*int(top_finger_body.index()):
                    log_wrapper.entries_per_body*int(top_finger_body.index())+3,:].T, 2)
p_MN_top = np.matmul(T(p_M_top), N_hat).flatten()
p_MT_top = np.matmul(T(p_M_top), T_hat).flatten()
p_MX_top = p_M_top[:,0].flatten()

In [ ]:
p_M_bot = np.expand_dims(logger.data()[log_wrapper.entries_per_body*int(bot_finger_body.index()):
                    log_wrapper.entries_per_body*int(bot_finger_body.index())+3,:].T, 2)
p_MN_bot = np.matmul(T(p_M_bot), N_hat).flatten()
p_MT_bot = np.matmul(T(p_M_bot), T_hat).flatten()
p_MX_bot = p_M_bot[:,0].flatten()

In [ ]:
v_M_top = np.expand_dims(
    logger.data()[log_wrapper.entries_per_body*int(top_finger_body.index())+6:
                  log_wrapper.entries_per_body*int(top_finger_body.index())+6+3].T,
    2)
v_MN_top = np.matmul(T(v_M_top), N_hat).flatten()
v_MT_top = np.matmul(T(v_M_top), T_hat).flatten()
v_MX_top = v_M_top[:,0].flatten()

In [ ]:
v_M_bot = np.expand_dims(
    logger.data()[log_wrapper.entries_per_body*int(bot_finger_body.index())+6:
                  log_wrapper.entries_per_body*int(bot_finger_body.index())+6+3].T,
    2)
v_MN_bot = np.matmul(T(v_M_bot), N_hat).flatten()
v_MT_bot = np.matmul(T(v_M_bot), T_hat).flatten()
v_MX_bot = v_M_bot[:,0].flatten()

#### Manipulator eq. terms

In [ ]:
J_top_no_interp = np.array(fold_ctrl.debug['J_top'])
J_top = scipy.interpolate.interp1d(fold_ctrl.debug['times'], J_top_no_interp, axis=0)(l_times)

In [ ]:
J_bot_no_interp = np.array(fold_ctrl.debug['J_bot'])
J_bot = scipy.interpolate.interp1d(fold_ctrl.debug['times'], J_bot_no_interp, axis=0)(l_times)

In [ ]:
q = np.expand_dims((logger.data()[log_wrapper.state_start_idx:log_wrapper.state_start_idx + 7,:]).T, 2)
d_q = np.expand_dims((logger.data()[log_wrapper.state_start_idx + 7:log_wrapper.state_start_idx + 14,:]).T, 2)

In [ ]:
M_no_interp = np.array(fold_ctrl.debug['M'])
M = scipy.interpolate.interp1d(fold_ctrl.debug['times'], M_no_interp, axis=0)(l_times)

In [ ]:
Cv_no_interp = np.array(fold_ctrl.debug['C']) # TODO fix this name
Cv = np.expand_dims(scipy.interpolate.interp1d(fold_ctrl.debug['times'], Cv_no_interp, axis=0)(l_times), 2)

#### Body derived terms
Meaning terms derived from the positions/velocities of bodies

In [ ]:
R = np.zeros((len(l_times), 3, 3))
R[:,1,1] = np.cos(theta_L)
R[:,1,2] = -np.sin(theta_L)
R[:,2,1] = np.sin(theta_L)
R[:,2,2] = np.cos(theta_L)

In [ ]:
T_hat_geo = np.matmul(R, y_hat)
N_hat_geo = np.matmul(R, z_hat)

In [ ]:
# Link edge
p_LE = p_L + (w_L/2)*T_hat-(h_L/2)*N_hat

In [ ]:
omega_vec_L = np.expand_dims(np.vstack((d_theta_L, np.zeros_like(d_theta_L), np.zeros_like(d_theta_L))).T, 2)
omega_vec_M = np.expand_dims(logger.data()[
        log_wrapper.entries_per_body*int(top_finger_body.index()) + 6 + 3:
        log_wrapper.entries_per_body*int(top_finger_body.index()) + 6 + 6
    ].T, 2)

In [ ]:
J_dot_top = diff_vec(J_top)
J_dot_bot = diff_vec(J_bot)

#### Contact terms
TODO: edit out/simplify eventually

In [ ]:
contact_point = logger.data()[log_wrapper.top_contact_entry_start_idx+11:log_wrapper.top_contact_entry_start_idx+14,:]

In [ ]:
p_C = np.expand_dims(contact_point.T, 2)
p_CT = np.matmul(T(p_C), T_hat).flatten()
p_CN = np.matmul(T(p_C), N_hat).flatten()

In [ ]:
r = np.linalg.norm(p_C - p_M_top, axis=1).flatten()

In [ ]:
# Assume all the slip speeds are the same
# TODO: verify
slip_speed = logger.data()[log_wrapper.top_contact_entry_start_idx+10,:]

In [ ]:
# d vector
d_vec = p_C - p_LE
d_T = np.matmul(T(d_vec), T_hat).flatten()
d_N = np.matmul(T(d_vec), N_hat).flatten()
d_X = d_vec[:,0].flatten()

In [ ]:
d_d_T = -d_theta_L*h_L/2-d_theta_L*r - v_LT + v_MT_top + d_theta_L*d_N
d_d_N = -d_theta_L*w_L/2-v_LN+v_MN_top-d_theta_L*d_T
d_d_X = diff_arr(d_X)

In [ ]:
dd_d_T = diff_arr(d_d_T)
dd_d_N = diff_arr(d_d_N)
dd_d_X = diff_arr(d_d_X)

In [ ]:
d_d_vec = diff_vec(d_vec)
d_d_vec_T = np.matmul(T(d_d_vec), T_hat).flatten()
d_d_vec_N = np.matmul(T(d_d_vec), N_hat).flatten()

In [ ]:
p_MConM_top = p_C - p_M_top
p_LConL = p_C - p_L

v_WConM_top = v_M_top + np.cross(omega_vec_M, p_MConM_top, axis=1)
v_WConL = v_L + np.cross(omega_vec_L, p_LConL, axis=1)

v_S_raw = v_WConM_top - v_WConL
v_S_N = np.matmul(N_hat_proj_mat, v_S_raw)
v_S = v_S_raw - v_S_N
v_S = v_S_raw - v_S_N
s_S = np.linalg.norm(v_S, axis=1).flatten()

In [ ]:
s_hat = v_S/np.expand_dims(np.linalg.norm(v_S, axis=1), 2)
s_hat_X = s_hat[:,0].flatten()
s_hat_T = np.matmul(T(s_hat), T_hat).flatten()

In [ ]:
d_d_T_diff = diff_arr(d_T)
d_d_N_diff = diff_arr(d_N)

### Friction coeficients

In [ ]:
mu_S = stribeck(1, 1, slip_speed/v_stiction)

## Outputs
These should all come directly from simulation, except for control forces.
### Measured accelerations ($a_{LT}, a_{LN}, a_{MT}, a_{MN}, \ddot\theta$)

In [ ]:
a_L_raw = logger.data()[log_wrapper.entries_per_body*paper.link_idxs[-1] + 12:
                     log_wrapper.entries_per_body*paper.link_idxs[-1]+12+3,:]
a_L = np.expand_dims(a_L_raw.T, 2)
a_LT = np.matmul(T(a_L), T_hat).flatten()
a_LN = np.matmul(T(a_L), N_hat).flatten()

In [ ]:
a_M_raw_top = logger.data()[log_wrapper.entries_per_body*int(top_finger_body.index()) + 12+3:
                     log_wrapper.entries_per_body*int(top_finger_body.index())+12+6,:]
a_M_top = np.expand_dims(a_M_raw_top.T, 2)
a_MT_top = np.matmul(T(a_M_top), T_hat).flatten()
a_MN_top = np.matmul(T(a_M_top), N_hat).flatten()
a_MX_top = a_M_top[:,0].flatten()
a_MY_top = a_M_top[:,1].flatten()
a_MZ_top = a_M_top[:,2].flatten()

In [ ]:
a_M_raw_bot = logger.data()[log_wrapper.entries_per_body*int(bot_finger_body.index()) + 12+3:
                     log_wrapper.entries_per_body*int(bot_finger_body.index())+12+6,:]
a_M_bot = np.expand_dims(a_M_raw_bot.T, 2)
a_MT_bot = np.matmul(T(a_M_bot), T_hat).flatten()
a_MN_bot = np.matmul(T(a_M_bot), N_hat).flatten()
a_MX_bot = a_M_bot[:,0].flatten()

In [ ]:
dd_theta_L = logger.data()[log_wrapper.entries_per_body*paper.link_idxs[-1] + 12 + 3,:]
dd_theta_M = logger.data()[log_wrapper.entries_per_body*int(top_finger_body.index()) + 12 + 3,:]

In [ ]:
dd_q = np.expand_dims((logger.data()[log_wrapper.gen_accs_start_idx:log_wrapper.gen_accs_start_idx+7,:]).T, 2)

### Contact forces ($F_{NL}, F_{FL}, F_{NM}, F_{FM}$)
Slightly different then just taking normal/tangent projections.

In [ ]:
# Top index
F_NM_top = np.zeros(len(l_times))
F_FMT_top = np.zeros(len(l_times))
F_FMX_top = np.zeros(len(l_times))
tau_contact = np.zeros((len(l_times), fold_ctrl.nq_arm, 1))
for i in range(log_wrapper.max_contacts):
    # Pull out terms
    start_idx = log_wrapper.top_contact_entry_start_idx + log_wrapper.entries_per_contact*i
    contact_point = np.expand_dims(logger.data()[start_idx+11:start_idx+14,:].T, 2)
    if np.all(np.isnan(contact_point)):
        break

    # Calculate intermediautes
    lever_arm = contact_point - p_M_top
    force = np.expand_dims(logger.data()[start_idx+0:start_idx+3,:].T, 2)
    torque = np.cross(lever_arm, force, axis=1)
    wrench = np.hstack((force, torque))
    
    # Calculate updates
    F_FMT = np.matmul(T(force), T_hat).flatten()
    F_NM = np.matmul(T(force), N_hat).flatten()
    F_FMX = force[:,0].flatten()
    
    # Update
    tau_contact += np.matmul(T(J_top), wrench)
    F_FMT_top += F_FMT
    F_NM_top += F_NM
    F_FMX_top += F_FMX

In [ ]:
# Bottom index
F_NM_bot = np.zeros(len(l_times))
F_FMT_bot = np.zeros(len(l_times))
F_FMX_bot = np.zeros(len(l_times))
for i in range(log_wrapper.max_contacts):
    # Pull out terms
    start_idx = log_wrapper.bot_contact_entry_start_idx + log_wrapper.entries_per_contact*i
    contact_point = np.expand_dims(logger.data()[start_idx+11:start_idx+14,:].T, 2)
    if np.all(np.isnan(contact_point)):
        break

    # Calculate intermediautes
    lever_arm = contact_point - p_M_bot
    force = np.expand_dims(logger.data()[start_idx+0:start_idx+3,:].T, 2)
    torque = np.cross(lever_arm, force, axis=1)
    wrench = np.hstack((force, torque))
    
    # Calculate updates
    F_FMT = np.matmul(T(force), T_hat).flatten()
    F_NM = np.matmul(T(force), N_hat).flatten()
    F_FMX = force[:,0].flatten()
    
    # Update
    tau_contact += np.matmul(T(J_bot), wrench)
    F_FMT_bot += F_FMT
    F_NM_bot += F_NM
    F_FMX_bot += F_FMX

In [ ]:
tau_contact = np.nan_to_num(tau_contact)
F_FMT_bot = np.nan_to_num(F_FMT_bot)
F_FMX_bot = np.nan_to_num(F_FMX_bot)
F_NM_bot = np.nan_to_num(F_NM_bot)
F_FMT_top = np.nan_to_num(F_FMT_top)
F_FMX_top = np.nan_to_num(F_FMX_top)
F_NM_top = np.nan_to_num(F_NM_top)

In [ ]:
F_FLT = -(F_FMT_bot*0+F_FMT_top)
F_FLX = -(F_FMX_bot*0+F_FMX_top)
F_NL = -(F_NM_bot*0+F_NM_top)

In [ ]:
# Equivalent contact forces for each object
F_contact_M_top = np.expand_dims(F_FMX_top, [1, 2])*x_hat + \
                  np.expand_dims(F_FMT_top, [1, 2])*T_hat + \
                  np.expand_dims(F_NM_top, [1, 2])*N_hat

In [ ]:
F_contact_M_bot = np.expand_dims(F_FMX_bot, [1, 2])*x_hat + \
                  np.expand_dims(F_FMT_bot, [1, 2])*T_hat + \
                  np.expand_dims(F_NM_bot, [1, 2])*N_hat

In [ ]:
F_contact_L = np.expand_dims(F_FLX, [1, 2])*x_hat + \
              np.expand_dims(F_FLT, [1, 2])*T_hat + \
              np.expand_dims(F_NL, [1, 2])*N_hat

### Object forces ($F_{ON}, F_{OT}, \tau_O$)

In [ ]:
joint_force_in_compliance_frame = logger.data()[
    log_wrapper.joint_entry_start_idx+6*(len(paper.joints) - 1):
    log_wrapper.joint_entry_start_idx+6*(len(paper.joints) - 1)+3,:]

In [ ]:
# Copy makes it mutable
F_OX = joint_force_in_compliance_frame[0,:].copy()
F_OT = joint_force_in_compliance_frame[1,:].copy()
F_ON = joint_force_in_compliance_frame[2,:].copy()

In [ ]:
joint_torque_in_compliance_frame = logger.data()[
    log_wrapper.joint_entry_start_idx+6*(len(paper.joints) - 1)+3:
    log_wrapper.joint_entry_start_idx+6*(len(paper.joints) - 1)+6,:]

In [ ]:
# Copy makes it mutable
tau_O = joint_torque_in_compliance_frame[0,:].copy()

In [ ]:
F_OX = np.nan_to_num(F_OX)
F_OT = np.nan_to_num(
F_OT)
F_ON = np.nan_to_num(
F_ON)
tau_O = np.nan_to_num(
tau_O)

## Control terms

In [ ]:
tau_ctrl_no_interp = np.array(fold_ctrl.debug['tau_ctrl'])
tau_ctrl = np.expand_dims(scipy.interpolate.interp1d(fold_ctrl.debug['times'], tau_ctrl_no_interp, axis=0)(l_times), 2)

In [ ]:
tau_out_no_interp = np.array(fold_ctrl.debug['tau_out'])
tau_out = np.expand_dims(scipy.interpolate.interp1d(fold_ctrl.debug['times'], tau_out_no_interp, axis=0)(l_times), 2)

In [ ]:
F_CX_no_interp = np.array(debug['F_CXs'])
F_CX = scipy.interpolate.interp1d(d_times, F_CX_no_interp, axis=0)(l_times)
F_CT_no_interp = np.array(debug['F_CTs'])
F_CT = scipy.interpolate.interp1d(d_times, F_CT_no_interp, axis=0)(l_times)
F_CN_no_interp = np.array(debug['F_CNs'])
F_CN = scipy.interpolate.interp1d(d_times, F_CN_no_interp, axis=0)(l_times)

In [ ]:
F_control = (
    np.expand_dims(F_CT, [1,2])*T_hat
    +
    np.expand_dims(F_CN, [1,2])*N_hat)
F_control[:,0,:] = np.expand_dims(F_CX, 1)

In [ ]:
# TODO: remove this
# tau_control_no_grav_comp = np.matmul(JT_trn, F_control)

In [ ]:
# tau_control = tau_control_no_grav_comp - tau_g

### Other outputs

In [ ]:
mu_hats = np.interp(l_times, d_times, debug['mu_ests'])

In [ ]:
forces_found = logger.data()[-1,:]

In [ ]:
in_contact_no_interp = np.array(debug['in_contact'])
in_contact = scipy.interpolate.interp1d(fold_ctrl.debug['times'], in_contact_no_interp)(l_times)

In [ ]:
raw_in_contact_no_interp = np.array(debug['raw_in_contact'])
raw_in_contact = scipy.interpolate.interp1d(fold_ctrl.debug['times'], raw_in_contact_no_interp)(l_times)

In [ ]:
t = l_times

# Plotting settings

In [ ]:
lw = 5

## Setting up time

In [ ]:
idx_start = np.argmax(in_contact > 0)
t_start = t[idx_start]
t_start

In [ ]:
first_contact_idx = np.argmax(in_contact)
last_contact_idx = in_contact.size - np.argmax(raw_in_contact[::-1]-1)-1

In [ ]:
last_not_in_contact_idx = in_contact.size - np.argmax(np.logical_not(raw_in_contact[::-1])-1)-1
t_start_of_last_contact = 0.14 #t[last_not_in_contact_idx+1]

In [ ]:
t_start = 0.01 #t[first_contact_idx] + 0.01
t_end = 0.15 # t[last_contact_idx]
idx_start = np.argmax(l_times > t_start)
idx_end = np.argmax(l_times > t_end)
if idx_end == 0:
    idx_end = len(l_times) - 1

## Defining functions

In [ ]:
def plot_eq(lhs, lhs_label, rhs, rhs_label, lhs_alpha=1, rhs_alpha=1, lw=5, finish_plotting=True, 
            plot_diff=True, figsize_=None, plot_err_as_pct=False, t_start=t_start, newline_in_title=False):
    if figsize_ is not None:
        plt.figure(figsize=figsize_)
    else:
        plt.figure(figsize=default_figsize)
    if isinstance(lhs, float) or isinstance(lhs, int):
        lhs = np.ones_like(l_times)*lhs
    if isinstance(rhs, float) or isinstance(rhs, int):
        rhs = np.ones_like(l_times)*rhs
    idx_start = np.argmax(l_times > t_start)
        
    b = np.broadcast(lhs, rhs)
    if np.count_nonzero(np.array(b.shape) > 1000) > 1:
        raise ValueError("Broadcast shape of lhs and rhs is {}, which is too large.".format(b.shape))
        
    plt.subplot(211)
    error = lhs[idx_start:idx_end]-rhs[idx_start:idx_end]
    if plot_err_as_pct:
        lhs_range = np.nanmax(lhs[idx_start:idx_end]) - np.nanmin(lhs[idx_start:idx_end])
        rhs_range = np.nanmax(rhs[idx_start:idx_end]) - np.nanmin(rhs[idx_start:idx_end])
        
        lhs_max_val = np.nanmax(np.abs(lhs[idx_start:idx_end]))
        rhs_max_val = np.nanmax(np.abs(rhs[idx_start:idx_end]))
        error /= np.nanmax([lhs_max_val, rhs_max_val])
        error *= 100
        
    plt.plot(l_times[idx_start:idx_end],
            error,
            linewidth=lw)
    plt.autoscale(enable=True, axis='x', tight=True)
    plt.xlabel("Time (s)")
    if plot_err_as_pct:
        plt.ylabel("Error (percent of max val)")
    else:
        plt.ylabel("Error")
    if newline_in_title:
        plt.title("Difference between {}\n and {}".format(lhs_label, rhs_label))
    else:
        plt.title("Difference between {} and {}".format(lhs_label, rhs_label))
    
    
    plt.subplot(212)
    plt.plot(l_times[idx_start:idx_end],
             lhs[idx_start:idx_end],
             label=lhs_label, linewidth=lw, color='lightskyblue', zorder=0, alpha=lhs_alpha)
    plt.plot(l_times[idx_start:idx_end],
             rhs[idx_start:idx_end],
             label=rhs_label, linewidth=lw, color='b', linestyle='--', zorder=1, alpha=rhs_alpha)
    plt.autoscale(enable=True, axis='x', tight=True)
    
    
#     plt.axvline(t[first_contact_idx], color='green', linewidth=lw)
#     plt.axvline(t_lose_contact, color='red', linewidth=lw)
    
    plt.legend()
    plt.xlabel("Time (s)")
    
    if finish_plotting:
        plt.show()

In [ ]:
errors = []
max_errors = []
med_errors = []
error_labels = []

In [ ]:
def update_error(lhs, lhs_label, rhs, rhs_label, newline_in_label=False):
    error = ((lhs) - (rhs))[idx_start:idx_end]
    
    label = r"$\left(" + lhs_label.replace("$", "") + r"\right) "
    if newline_in_label:
        label += "$\n$"
    label += "- \left(" + rhs_label.replace("$", "") + r"\right)$"
    label_idx = None
    for i, l in enumerate(error_labels):
        if l == label:
            label_idx = i
            break
    max_error = np.nanmax(np.abs(error))
    med_error = np.median(np.abs(error[np.logical_not(np.isnan(error))]))
    if label_idx is None:
        errors.append(error)
        error_labels.append(label)
        max_errors.append(max_error)
        med_errors.append(med_error)
    else:
        errors[label_idx] = error
        max_errors[label_idx] = max_error
        med_errors[label_idx] = med_error

# Plots (environmental constraints)
There are things where I expect them to be satisfied no matter what, no matter the succes my controller.

## Link (T) accel $m_La_{LT} = F_{FLT} + F_{GT} +F_{OT}$
*If this plot is correct:*
The free body diagram is correct in thet $\hat T$ direction on the link.

In [ ]:
lhs_label = r'$m_La_{LT}$'
rhs_label = r'$F_{FLT} + F_{GT} +F_{OT}$'

lhs = m_L*a_LT

rhs = F_FLT+F_GT+F_OT

plot_eq(lhs, lhs_label, rhs, rhs_label)
update_error(lhs, lhs_label, rhs, rhs_label)

## Link (N) accel $m_La_{LN} = F_{NL} + F_{GN} +F_{ON}$
*If this plot is correct:*
The free body diagram is correct in thet $\hat N$ direction on the link.

In [ ]:
lhs_label = r'$m_La_{LN}$'
rhs_label = r'$F_{NL} + F_{GN} +F_{ON}$'

lhs = m_L*a_LN

rhs = F_NL + F_GN + F_ON

plot_eq(lhs, lhs_label, rhs, rhs_label)
update_error(lhs, lhs_label, rhs, rhs_label)

## Link (x) angle accel $I_L\ddot\theta_L = -\frac{w_L}{2}F_{ON} - (p_{CN}-p_{LN})F_{FL} + (p_{CT}-p_{LT})F_{NL}+\tau_O$
*If this plot is correct:* The moment balance on the link is correct.

In [ ]:
lhs_label = r'$I_L\ddot\theta$'
rhs_label = r'$-\frac{w_L}{2}F_{ON} - (p_{CN}-p_{LN})F_{FL} + (p_{CT}-p_{LT})F_{NL}+\tau_O$'

lhs = I_L*dd_theta_L

N_lever_arm = np.nan_to_num(p_CN-p_LN)
T_lever_arm = np.nan_to_num(p_CT-p_LT)
rhs = (-w_L/2)*F_ON - N_lever_arm * F_FLT + T_lever_arm*F_NL + tau_O

plot_eq(lhs, lhs_label, rhs, rhs_label)
update_error(lhs, lhs_label, rhs, rhs_label)

## 3rd law normal forces $F_{NL} = -F_{NM}$
*If this plot is correct:* Newton's 3rd law is correction applied to the normal forces.

In [ ]:
lhs_label = r'$F_{NL}$'
rhs_label = r'$-F_{NM_{top}}-F_{NM_{bot}}$'

lhs = F_NL

rhs = -F_NM_top - F_NM_bot

plot_eq(lhs, lhs_label, rhs, rhs_label)
update_error(lhs, lhs_label, rhs, rhs_label)

## 3rd law friction forces $F_{FLT} = -F_{FMT}$
*If this plot is correct:* Newton's 3rd law is correction applied to the friction forces in the tangent direction.

In [ ]:
lhs_label = r'$F_{FLT}$'
rhs_label = r'$-F_{FMT_{top}}-F_{FMT_{bot}}$'

lhs = F_FLT

rhs = -F_FMT_top - F_FMT_bot

plot_eq(lhs, lhs_label, rhs, rhs_label)
update_error(lhs, lhs_label, rhs, rhs_label)

## d vec tangential derivative $-\ddot\theta_L h_L/2 - \ddot\theta_L r + \dot\theta_L^2 w_L/2 - a_{LT} + a_{MT} = -\ddot\theta_L {d}_N + \ddot{d}_T - \dot\theta_L^2 {d}_T - 2\dot\theta_L \dot{d}_N$
*If this plot is correct:* The derivatives for  $\vec d$  are correct.

In [ ]:
lhs = -dd_theta_L*(h_L/2+r) + d_theta_L**2*w_L/2 - a_LT + a_MT_top
lhs_label = r"$-\ddot\theta_L h_L/2 - \ddot\theta_L r + \dot\theta_L^2 w_L/2 - a_{LT} + a_{MT}$"

rhs = -dd_theta_L*d_N + dd_d_T - d_theta_L**2*d_T - 2*d_theta_L*d_d_N
rhs_label = r"$-\ddot\theta_L {d}_N + \ddot{d}_T - \dot\theta_L^2 {d}_T - 2\dot\theta_L \dot{d}_N$"

plot_eq(lhs, lhs_label, rhs, rhs_label)
update_error(lhs, lhs_label, rhs, rhs_label, newline_in_label=True)

## d vec normal derivative $- \frac{w_{L}}{2}\ddot\theta_{L} - \frac{h_{L}}{2}\dot\theta_{L}^{2} - \dot\theta_{L}^{2} r - a_{LN} + a_{MN} = \ddot\theta_{L} d_{T} + \ddot{d}_N - \dot\theta_{L}^{2} d_{N} + 2 \dot\theta_{L} \dot{d}_T$
*If this plot is correct:* The derivatives for  $\vec d$  are correct.

In [ ]:
lhs = -dd_theta_L*w_L/2 - d_theta_L**2*h_L/2 - d_theta_L**2*r - a_LN + a_MN_top
lhs_label = r"$-\ddot\theta_L w_L/2 - \dot\theta_L^2 h_L/2 - \dot\theta_L^2 r - a_{LN} + a_{MN}$ (differentiating geometry)"

rhs = dd_theta_L*d_T + dd_d_N - d_theta_L**2*d_N + 2*d_theta_L*d_d_T
rhs_label = r"$\ddot\theta_L {d}_T + \ddot{d}_N - \dot\theta_L^2 {d}_N + 2 \dot\theta_L \dot{d}_T$ (differentiating vector)"

plot_eq(lhs, lhs_label, rhs, rhs_label, newline_in_title=True)

lhs_label = r"$-\ddot\theta_L w_L/2 - \dot\theta_L^2 h_L/2 - \dot\theta_L^2 r - a_{LN} + a_{MN}$"
rhs_label = r"$\ddot\theta_L {d}_T + \ddot{d}_N - \dot\theta_L^2 {d}_N + 2 \dot\theta_L \dot{d}_T$"
update_error(lhs, lhs_label, rhs, rhs_label, newline_in_label=True)
plt.show()

## No penetration $\ddot d_N = 0$
*If this plot is correct:* There is effectively no penetration between the rigid bodies.

In [ ]:
lhs_label = r'$\ddot{d}_N$'
rhs_label = r'0'

lhs = dd_d_N

rhs = 0

plot_eq(lhs, lhs_label, rhs, rhs_label)
update_error(lhs, lhs_label, rhs, rhs_label)

## Link (T) friction equation $F_{FLT}=\mu\mu_SF_{NL}\hat s_T$

In [ ]:
lhs_label = r'$\mu\mu_SF_{NL}\hat s_T$'
rhs_label = r'$F_{FLT}$'

lhs = np.nan_to_num(mu_S*F_NL*mu*s_hat_T)
rhs = F_FLT

plot_eq(lhs, lhs_label, rhs, rhs_label)
update_error(lhs, lhs_label, rhs, rhs_label)

## Link (x) friction equation $F_{FLX}=\mu\mu_SF_{NL}\hat s_X$

In [ ]:
lhs_label = r'$\mu\mu_SF_{NL}\hat s_X$'
rhs_label = r'$F_{FLX}$'

lhs = np.nan_to_num(mu_S*F_NM*mu*s_hat_X)
rhs = F_FMX

plot_eq(lhs, lhs_label, rhs, rhs_label)
update_error(lhs, lhs_label, rhs, rhs_label)

## Top Jacobian and (x) accel $a_{MX_{top}}=\left(\dot{J}_{top} \dot{\vec{q}} + J_{top}\ddot{\vec{q}}\right)_3$

In [ ]:
lhs_label = r'$a_{MX_{top}}$'
rhs_label = r'$\left(\dot{J}_{top} \dot{\vec{q}} + J_{top}\ddot{\vec{q}}\right)_0$'

lhs = a_MX_top
rhs = (np.matmul(J_dot_top, d_q) + np.matmul(J_top, dd_q))[:,0].flatten()

plot_eq(lhs, lhs_label, rhs, rhs_label)
update_error(lhs, lhs_label, rhs, rhs_label)

In [ ]:
lhs_label = r'$a_{MY_{top}}$'
rhs_label = r'$\left(\dot{J}_{top} \dot{\vec{q}} + J_{top}\ddot{\vec{q}}\right)_1$'

lhs = a_MY_top
rhs = (np.matmul(J_dot_top, d_q) + np.matmul(J_top, dd_q))[:,1].flatten()

plot_eq(lhs, lhs_label, rhs, rhs_label)
update_error(lhs, lhs_label, rhs, rhs_label)

In [ ]:
lhs_label = r'$a_{MZ_{top}}$'
rhs_label = r'$\left(\dot{J}_{top} \dot{\vec{q}} + J_{top}\ddot{\vec{q}}\right)_2$'

lhs = a_MZ_top
rhs = (np.matmul(J_dot_top, d_q) + np.matmul(J_top, dd_q))[:,2].flatten()

plot_eq(lhs, lhs_label, rhs, rhs_label)
update_error(lhs, lhs_label, rhs, rhs_label)

## Manipulator equation $M \ddot q + C \dot q = \tau_g + \vec \tau_{\text{contact}} + \vec \tau_{\text{control}}$

In [ ]:
lhs_ = np.matmul(M, dd_q) + Cv
lhs_label_ = r"M \ddot q + C \dot q"
rhs_ = tau_g + tau_contact + tau_out
rhs_label_ = r"\tau_g + \vec \tau_{contact} + \vec \tau_{control}"
labels = [str(i) for i in range(7)]

for i, label in enumerate(labels):
    lhs = lhs_[:,i,:]
    lhs_label = r"$\left(" + lhs_label_ + r"\right)_" + label + "$"

    rhs = rhs_[:,i,:]
    rhs_label = r"$\left(" + rhs_label_ + r"\right)_" + label + "$"

    plot_eq(lhs, lhs_label, rhs, rhs_label)
    plt.show()

# Plot errors
How accurate is my modeling?

In [ ]:
plt.figure(figsize=(16,16))
plt.barh(error_labels[::-1], max_errors[::-1], label="Maximum error")
plt.barh(error_labels[::-1], med_errors[::-1], label="Median error")
plt.axvline(1e-5, color='k', linestyle='--', label="1e-5")
plt.xscale("log")
plt.xlabel("Log(error)")
plt.ylabel("Equation")
plt.legend()
plt.xlim(10**-16, 10**3)
plt.show()

# Other plots
These aren't necessarily constraints in the program, but I do want to check them.

## Friction
### Norm

In [ ]:
lhs = np.linalg.norm(np.expand_dims(F_FLX, [1,2])*x_hat + np.expand_dims(F_FLT, [1,2])*T_hat, axis=1)
lhs_label = r"$||F_{FLT} + F_{FLX}||$"

rhs = np.abs(np.expand_dims(mu*mu_S*F_NL, 1))
rhs_label = "$|\mu\mu_{S}F_{NL}|$"

plot_eq(lhs, lhs_label, rhs, rhs_label)
plt.show()

In [ ]:
lhs = s_S
lhs_label = r"Slip speed candidate norm"

rhs = slip_speed
rhs_label = "Actual slip speed"

plot_eq(lhs, lhs_label, rhs, rhs_label)
plt.show()

## Direction

In [ ]:
F_hat = np.expand_dims(F_FLT, [1,2])*T_hat + np.expand_dims(F_FLX, [1,2])*x_hat
F_hat = F_hat/np.expand_dims(np.linalg.norm(F_hat, axis=1), 2)

In [ ]:
sig1 = s_hat
sig1_label = r"$\hat s$"

sig2 = F_hat
sig2_label = r"$\hat F$"
plot_vec(sig1, sig1_label, sig2, sig2_label, start_time=t_start_of_last_contact)

In [ ]:
sig1 = s_hat
sig1_label = r"$\hat s$"

sig2 = F_hat
sig2_label = r"$\hat F$"
plot_vec_XTN(sig1, sig1_label, sig2, sig2_label, start_time=t_start_of_last_contact)

## Actual values

In [ ]:
F_mag = mu_S*F_NL*mu

In [ ]:
F_F_est = s_hat*np.expand_dims(np.vstack((F_mag,F_mag,F_mag)).T, 2)

In [ ]:
sig1 = F_F_est
sig1_label = r"Estimated friction"

sig2 = F_contact_L - N_hat_proj_mat@F_contact_L
sig2_label = r"Actual friction"
plot_vec_XTN(sig1, sig1_label, sig2, sig2_label, start_time=t_start_of_last_contact)

## Plots (control targets)
These will only be satisfied if the controller is successful.

### $\ddot{d}_N = \ddot{d}_{Nd}$

In [ ]:
lhs_label = r'$\ddot{d}_N$'
rhs_label = r'$\ddot{d}_{Nd}$'

lhs = dd_d_N

rhs = 0

plot_eq(lhs, lhs_label, rhs, rhs_label)

### $a_{LN} = a_{LNd}$

In [ ]:
lhs_label = r'$a_{LN}$'
rhs_label = r'$a_{LNd}$'

lhs = a_LN

rhs = fold_ctrl.a_LNd

plot_eq(lhs, lhs_label, rhs, rhs_label)

In [ ]:
plot_eq(d_T, "$d_T$", fold_ctrl.d_Td, "$d_{Td}$")

In [ ]:
plot_eq(d_X, "$d_X$", 0, "0")

In [ ]:
lw=5
plt.figure(figsize=(16, 8))
plt.axhline(fold_ctrl.d_Td, linestyle='--', color='gray', linewidth=lw, label="Desired $d_T$")
plt.plot(l_times, d_T, label='$d_T$', linewidth=lw)
plt.axhline(-(0), label='Bounds on $d_T$ (0 and link width)', linestyle='--', color='k', linewidth=lw)
plt.axhline(-(paper.link_width), linestyle='--', color='k', linewidth=lw)
plt.legend()
plt.xlabel("Time (seconds)")
plt.ylabel("Distance (m)")
plt.autoscale(enable=True, axis='x', tight=True)
plt.xlim(t_start, t_end)
plt.show()

# Control estimates

In [ ]:
F_OT_debug = scipy.interpolate.interp1d(d_times, debug['F_OTs'], axis=0)(l_times)
F_ON_debug = scipy.interpolate.interp1d(d_times, debug['F_ONs'], axis=0)(l_times)
tau_O_debug = scipy.interpolate.interp1d(d_times, debug['tau_Os'], axis=0)(l_times)

In [ ]:
plot_eq(F_OT_debug, "debug", F_OT, "actual")

In [ ]:
plot_eq(F_ON_debug, "debug", F_ON, "actual")

In [ ]:
plot_eq(tau_O_debug, "debug", tau_O, "actual")

# Parameter estimation plots
If we are using adaptive control, how well do we learn the parameters?

In [ ]:
plot_eq(mu_hats, "$\hat\mu$", constants.FRICTION, r"$\mu$")

# Other useful plots

## Friction plots

In [ ]:
lw=5
plt.figure(figsize=(16,8))
plt.plot(l_times[idx_start:idx_end],
         abs(F_FLT[idx_start:idx_end]), label=r"$\left|F_{FL}\right|$", linewidth=lw, color='lightskyblue', zorder=0)
plt.plot(l_times[idx_start:idx_end],
         abs(F_FMT[idx_start:idx_end]), label=r"$\left|F_{FM}\right|$", linewidth=lw, color='b', linestyle='--', zorder=1)
plt.plot(l_times[idx_start:idx_end],
         abs(F_NL[idx_start:idx_end])*constants.FRICTION, label=r"$\mu\left|F_{NL}\right|$", linewidth=lw, color='k', linestyle=':')

plt.autoscale(enable=True, axis='x', tight=True)
plt.legend()
plt.xlabel("Time")
plt.show()

In [ ]:
lw=5
plt.figure(figsize=(16, 8))
plt.plot(l_times[idx_start:idx_end], slip_speed[idx_start:idx_end], label="slip speed", linewidth=5)
plt.axhline(v_stiction, linestyle="--", color="k", label="$v_{stiction}$", linewidth=5)
plt.xlabel("Time (seconds)")
plt.ylabel("Speed (m/s)")
# plt.ylim(0, 2*v_stiction)
plt.autoscale(enable=True, axis='x', tight=True)
plt.legend()
plt.show()

## Joint torques

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(l_times[idx_start:idx_end], 
         joint_torque_in_compliance_frame[0,idx_start:idx_end], linewidth=5, color='r', label=r"\tau_x")
plt.plot(l_times[idx_start:idx_end], 
         joint_torque_in_compliance_frame[1,idx_start:idx_end], linewidth=5, color='g', label=r"\tau_y")
plt.plot(l_times[idx_start:idx_end], 
         joint_torque_in_compliance_frame[2,idx_start:idx_end], linewidth=5, color='b', label=r"\tau_z")
plt.autoscale(enable=True, axis='x', tight=True)

## Sliding surface variable

In [ ]:
s = fold_ctrl.lamda*(d_T - fold_ctrl.d_Td) + (d_d_T)

In [ ]:
plot_eq(s, "$s$", 0, "0")

# Debugging

In [ ]:
for a, b in fold_ctrl.contacts:
    name_a = plant.get_body(BodyIndex(a)).name()
    name_b = plant.get_body(BodyIndex(b)).name()
    print("{} in contact with {}".format(name_a, name_b))